-----

### **Course: Advanced Prompt Engineering for Banking**

### **Lab 6: Working with Open-Source Models via Groq and Hugging Face**

**Objective:** This lab will guide you through setting up and using powerful open-source models from providers like Groq (for speed) and Hugging Face (for variety). You will learn how to manage API keys and access tokens, create reusable functions to interact with these services, and apply the same prompting techniques to different underlying models.

-----

### **1. Setup and Connection Instructions**

First, we need to install the specific client libraries for Groq and Hugging Face.

#### **1.1 Installing Libraries**

In [ ]:
# Run this cell to install the necessary packages
!pip install groq==0.13.0
!pip install huggingface_hub==0.26.2

#### **1.2 Connecting to Groq Cloud** ☁️

To use Groq's Llama 3.2 model, you need a free API key.

**How to get your Groq API Key:**

1.  Go to the Groq Cloud console: `https://console.groq.com/`
2.  Sign up or log in to your account.
3.  In the left-hand menu, navigate to **API Keys**.
4.  Click the "Create API Key" button. Give it a name (e.g., "colab-notebook-key") and click "Create".
5.  **Important:** Copy the key immediately and save it somewhere secure. You will not be able to see it again after you close the window.

Now, run the following Python code to securely input your key.

In [ ]:
import os
# from getpass import getpass

# Securely get the Groq API key
# groq_api_key = getpass("Enter your Groq API Key: ")
# os.environ["GROQ_API_KEY"] = groq_api_key

#### **1.3 Connecting to Hugging Face 🤗**

To use models from Hugging Face, you need an Access Token. Many powerful models, including Meta's Llama models, are "gated" and require you to accept their terms before you can access them.

**How to get your Hugging Face Access Token:**

1.  Go to the Hugging Face website: `https://huggingface.co/`
2.  Sign up or log in to your account.
3.  Click on your profile picture in the top-right corner and go to **Settings**.
4.  In the left-hand menu, navigate to **Access Tokens**.
5.  Click the "New token" button. Give it a name and assign it a "write" role, then click "Generate a token".
6.  Copy the token and save it.

**How to Access Gated Models (like Llama 3.2):**
Before you can use a gated model via API, you must accept its terms on the website.

1.  Navigate to the model's page, for example: `https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct`
2.  You will see a prompt asking you to agree to the terms and share your information.
3.  Review the license and submit the form. Approval is often instant but can sometimes take a short while. **You must complete this step to avoid "access forbidden" errors.**

Now, run the following code to securely input your token.

In [ ]:
# from getpass import getpass

# Securely get the Hugging Face Access Token
# hf_token = getpass("Enter your Hugging Face Access Token: ")
# os.environ["HF_TOKEN"] = hf_token

-----

### **2. Creating Helper Functions**

Now, we'll create the two new execution functions as you specified. One for Groq and one for Hugging Face.

In [ ]:
from groq import Groq
from huggingface_hub import InferenceClient
from langchain_core.prompts import PromptTemplate
from IPython.display import Markdown, display

# Initialize clients
# groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# The HF client is initialized within the function as the model can change

def execute_prompt_groq(prompt_template, title, model_name="llama-3.2-90b-vision-preview", **kwargs):
    """
    Executes a prompt using the Groq API and prints the output.
    Uses a default model if none is provided.
    """
    print(f"--- {title} (Provider: Groq, Model: {model_name}) ---")

    prompt = PromptTemplate.from_template(prompt_template)
    final_prompt_text = prompt.format(**kwargs)
    print("PROMPT:")
    print(final_prompt_text)

    chat_completion = groq_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": final_prompt_text,
            }
        ],
        model=model_name,
        temperature=0.0,
    )

    response = chat_completion.choices[0].message.content
    print("\n" + "="*20 + " RESPONSE " + "="*20 + "\n")
    display(Markdown(response))
    print("\n" + "-"*50 + "\n")


def execute_prompt_hf(prompt_template, title, model_name, **kwargs):
    """
    Executes a prompt using the Hugging Face Inference API and prints the output.
    Requires the model name to be passed explicitly.
    """
    print(f"--- {title} (Provider: Hugging Face, Model: {model_name}) ---")

    # Initialize client with the specified model
    hf_client = InferenceClient(model=model_name, token=os.environ.get("HF_TOKEN"))

    prompt = PromptTemplate.from_template(prompt_template)
    final_prompt_text = prompt.format(**kwargs)
    print("PROMPT:")
    print(final_prompt_text)

    # The HF Inference Client uses a different format for messages
    response = hf_client.chat_completion(
        messages=[{"role": "user", "content": final_prompt_text}],
        max_tokens=500,
        temperature=0.1
    )

    print("\n" + "="*20 + " RESPONSE " + "="*20 + "\n")
    display(Markdown(response.choices[0].message.content))
    print("\n" + "-"*50 + "\n")

-----

### **3. Applying Prompting Techniques: Zero-Shot Classification**

Here is the implementation for the first use case. We will classify the sentiment of a financial news headline using both Groq and Hugging Face back-to-back.

**Scenario:** An automated system for a large investment bank needs to classify the sentiment of news headlines. The output must be a single, standardized word (`Positive`, `Negative`, `Neutral`).

In [ ]:
news_headline = "Supply chain disruptions in Asia are expected to cause a Q4 earnings dip for major tech stocks, warns analyst."

#### **Groq Implementation (Llama 3.2 90B)**

**Naive Prompt 👎**

In [ ]:
naive_prompt_1 = "What do you think the sentiment of this news headline is?\n\nHeadline: \"{headline}\""

execute_prompt_groq(
    naive_prompt_1,
    "1.1 Naive Sentiment Classification",
    headline=news_headline
)

**Improved Prompt 👍**

In [ ]:
improved_prompt_1 = """
Act as a financial market analyst AI.
Given the following news headline, classify the market sentiment.

The sentiment must be one of the following: Positive, Negative, or Neutral.
Return only the single-word sentiment category and nothing else.

Headline: "{headline}"
"""
execute_prompt_groq(
    improved_prompt_1,
    "1.2 Improved Zero-Shot Classification",
    headline=news_headline
)

#### **Hugging Face Implementation (Llama 3.2 1B)**

**Naive Prompt 👎**

In [ ]:
# We use the same naive prompt text, but call the Hugging Face function
execute_prompt_hf(
    naive_prompt_1,
    "1.3 Naive Sentiment Classification",
    model_name="meta-llama/Llama-3.2-1B-Instruct",
    headline=news_headline
)

**Improved Prompt 👍**

In [ ]:
# We use the same improved prompt text, but call the Hugging Face function
execute_prompt_hf(
    improved_prompt_1,
    "1.4 Improved Zero-Shot Classification",
    model_name="meta-llama/Llama-3.2-1B-Instruct",
    headline=news_headline
)

You can now use these two functions, `execute_prompt_groq` and `execute_prompt_hf`, to complete the remaining 12 use cases from our previous labs. Just remember to pass the `model_name` for the Hugging Face function calls.